In [3]:
import os 
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'scripts'))

import numpy as np
import seaborn as sns
from ipywidgets import interact
from wordcloud import WordCloud
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as mtick

import utils

data_file = "cars.csv"

PROJECT_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(PROJECT_DIR, 'data', 'csv')

DATA_FILE = os.path.join(DATA_DIR, data_file)

df = pd.read_csv(DATA_FILE)

df = utils.clean_data(df)
csv = df

In [4]:
csv["brand"] = csv["Name"].apply(lambda x: x.split(" ")[0])
csv['reference'] = csv['Name'].apply(lambda x: x.split(' ', 1)[1])
csv["Engine"] = csv["Engine"].str.replace(" CC", "").astype(float)
csv["Power"] = csv["Power"].str.replace(" bhp", "").astype(float)

#The Mileage is represented on kmlp
csv["Mileage"] = np.where(csv["Mileage"].str.split(" ", expand=True)[1] == "kmpl", 
        csv["Mileage"].str.split(" ", expand=True)[0].astype(float),  
        csv["Mileage"].str.split(" ", expand=True)[0].astype(float)*0.712/(1.60934*0.264172)
        )

csv["New_Price"].fillna("0 Lakh", inplace=True)
csv["New_Price"] = np.where(csv["New_Price"].str.split(" ", expand=True)[1] == "Cr",
        csv["New_Price"].str.split(" ", expand=True)[0].astype(float)*10000000,
        csv["New_Price"].str.split(" ", expand=True)[0].astype(float)*100000
        )

csv["Price"] = csv["Price"].astype(float)*100000

csv.drop(["Name", "Unnamed: 0"], axis=1, inplace=True)

In [5]:


csv_2 = csv.copy()
csv_2 = csv_2[csv_2["reference"]=="XUV500 W8 2WD"]

X = csv_2["Year"].values.reshape(-1, 1)
y = csv_2["Price"].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [8]:
csv_2.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,brand,reference
89,Delhi,2014.0,58000.0,Diesel,Manual,First,15.1,2179.0,140.0,7.0,NaN,810000.0,Mahindra,XUV500 W8 2WD
103,Delhi,2012.0,65000.0,Diesel,Manual,Second,16.0,2179.0,140.0,7.0,NaN,575000.0,Mahindra,XUV500 W8 2WD
155,Mumbai,2012.0,32792.0,Diesel,Manual,First,15.1,2179.0,140.0,7.0,NaN,790000.0,Mahindra,XUV500 W8 2WD
427,Kochi,2013.0,61672.0,Diesel,Manual,First,16.0,2179.0,140.0,7.0,NaN,753000.0,Mahindra,XUV500 W8 2WD
622,Delhi,2012.0,72450.0,Diesel,Manual,First,15.1,2179.0,140.0,7.0,NaN,810000.0,Mahindra,XUV500 W8 2WD
